In [5]:
import pymc3 as pm


You can find the C code in this temporary file: C:\Users\bakera\AppData\Local\Temp\theano_compilation_error_trvrj3lk


AttributeError: module 'theano' has no attribute 'gof'